In [2]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path
fig_path       = config.fig_path

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

area_sum = 19.925158940748766


Entered /tera11/zhwei/students/Xionghui/data/run/500/


In [2]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Sr', 'Sr'),
    ('Ssoil', 'Band1'),
    ('Area','area')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  

    # print(f"File: {file}, Variable: {variable_name}, Index: {index}")
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

# print((df.groupby('D_FM_median')['Area'].sum()/1e12)/16.092963536896)
area_sum = df['Area'].sum()/1e12
sbedrock_mean = df['Sbedrock'].mean()
sr_mean = df['Sr'].mean()
ssoil_mean = df['Ssoil'].mean()
print(area_sum)
print(sbedrock_mean)
print(sr_mean)
print(ssoil_mean)


df.drop(labels='mask123',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)

with open('csv/Global_S.csv','w') as f:
    df.to_csv(f)
df

19.925158940748766
130.46811
234.49243
93.050095


,lat,lon,Sbedrock,Sr,Ssoil,Area
709889414,-55.764582,-67.439577,7.599648,36.117283,0.000000,120773.879698
709889415,-55.764582,-67.435411,6.355453,35.986019,0.000000,120773.879698
709889416,-55.764582,-67.431244,5.111259,35.854755,120.396660,120773.879698
709889417,-55.764582,-67.427077,3.867064,35.723492,120.416367,120773.879698
709975814,-55.760415,-67.439577,7.970362,37.879093,0.000000,120786.785891
...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,39.297733,39.297733,0.000000,26309.225697
3586500783,82.960426,-26.735409,39.880871,39.880871,0.000000,26309.225697
3586500784,82.960426,-26.731242,40.464008,40.464008,0.000000,26309.225697
3586500785,82.960426,-26.727075,41.047146,41.047146,0.000000,26309.225697


In [3]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Dbedrock_Frequency', 'Dbedrock'),
    ('Area','area')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
# df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

df['Dbedrock_Frequency'] = df['Dbedrock_Frequency'].round().astype(int)

area_DF_sum = df['Area'].sum()/1e12

d_f = (df.groupby('Dbedrock_Frequency')['Area'].sum()/1e12)

print(area_DF_sum)
print(area_sum)
print(d_f)
print(d_f/area_DF_sum)

df.drop(labels='mask123',axis=1,inplace=True)
with open('csv/Global_DF.csv','w') as f:
    df.to_csv(f)
df

31.70627102151302
19.925158940748766
Dbedrock_Frequency
1     5.810638
2    13.486521
3    12.400530
4     0.008582
Name: Area, dtype: float64
Dbedrock_Frequency
1    0.183265
2    0.425358
3    0.391107
4    0.000271
Name: Area, dtype: float64


,lat,lon,Dbedrock_Frequency,Area
707038168,-55.902082,-67.631244,3,120347.617423
707124692,-55.897915,-67.114577,3,120360.544658
707124693,-55.897915,-67.110411,3,120360.544658
707124694,-55.897915,-67.106244,3,120360.544658
707124695,-55.897915,-67.102077,3,120360.544658
...,...,...,...,...
3586500782,82.960426,-26.739575,1,26309.225697
3586500783,82.960426,-26.735409,1,26309.225697
3586500784,82.960426,-26.731242,1,26309.225697
3586500785,82.960426,-26.727075,1,26309.225697


In [2]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),

    ('S_FD', 'FD'),
    ('S_FY', 'FD'),
    # ('D_FD_median', 'FD'),
    # ('D_FM_median', 'FD'),
    ('D_FD_mean', 'FD'),
    ('D_FM_mean', 'FD'),

    ('S_Duration', 'Duration'),
    # ('D_Duration_median', 'Duration'),
    ('D_Duration_mean', 'Duration'),

    ('Area', 'area')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

df['S_FD'] = df['S_FD'].round().astype(int)
df['S_FY'] = df['S_FY'].round().astype(int)
df['D_FD_mean'] = df['D_FD_mean'].round().astype(int)
df['D_FM_mean'] = df['D_FM_mean'].round().astype(int)
df['S_Duration'] = df['S_Duration'].round().astype(int)
df['D_Duration_mean'] = df['D_Duration_mean'].round().astype(int)


s_fd = (df.groupby('S_FD')['Area'].sum()/1e12)/area_sum
s_fy = (df.groupby('S_FY')['Area'].sum()/1e12)/area_sum
d_fd_mean = (df.groupby('D_FD_mean')['Area'].sum()/1e12)/area_sum
d_fm_mean = (df.groupby('D_FM_mean')['Area'].sum()/1e12)/area_sum
s_duration = (df.groupby('S_Duration')['Area'].sum()/1e12)/area_sum
d_duration_mean = (df.groupby('D_Duration_mean')['Area'].sum()/1e12)/area_sum

# print(s_fd)
print(s_fy)
# print(d_fd_mean)
print(d_fm_mean)
# print(s_duration)
# print(d_duration_mean)

print(s_fd[s_fd.index <= 1000].sum())
print(s_fy[s_fy.index <= 1].sum())
print(d_fd_mean[d_fd_mean.index <= 50].sum())
print(d_fm_mean[d_fm_mean.index <= 1].sum())
print(s_duration[s_duration.index >= 365].sum())
print(d_duration_mean[d_duration_mean.index >= 30].sum())

print((df['Area'].sum()/1e12)/area_sum)

df.drop(labels='mask123',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
with open('csv/Global_FDFMFY.csv','w') as f:
    df.to_csv(f)
df

S_FY
1     0.579630
2     0.122586
3     0.071316
4     0.049834
5     0.034922
6     0.026222
7     0.021504
8     0.027290
9     0.013594
10    0.015985
11    0.007034
12    0.006310
13    0.006906
14    0.004981
15    0.003664
16    0.003726
17    0.002652
18    0.001844
Name: Area, dtype: float64
D_FM_mean
1     0.275152
2     0.169267
3     0.121319
4     0.082524
5     0.114129
6     0.136029
7     0.075710
8     0.014688
9     0.010381
10    0.000741
11    0.000054
12    0.000004
Name: Area, dtype: float64
0.759101035698238
0.5796301315196667
0.39338094422462727
0.27515208296667815
0.5744805819330193
0.4449215967150624
1.0


,lat,lon,Sbedrock,S_FD,S_FY,D_FD_mean,D_FM_mean,S_Duration,D_Duration_mean,Area
709889414,-55.764582,-67.439577,7.599648,846,3,59,3,51,2,120773.879698
709889415,-55.764582,-67.435411,6.355453,806,3,57,3,48,2,120773.879698
709889416,-55.764582,-67.431244,5.111259,767,3,54,3,46,2,120773.879698
709889417,-55.764582,-67.427077,3.867064,728,2,51,3,43,2,120773.879698
709975814,-55.760415,-67.439577,7.970362,887,3,62,3,53,2,120786.785891
...,...,...,...,...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,39.297733,22,1,18,1,2742,152,26309.225697
3586500783,82.960426,-26.735409,39.880871,22,1,18,1,2740,152,26309.225697
3586500784,82.960426,-26.731242,40.464008,22,1,18,1,2738,152,26309.225697
3586500785,82.960426,-26.727075,41.047146,22,1,18,1,2736,152,26309.225697


In [8]:
print(s_fd[s_fd.index <= 1000].sum())
print(s_fy[s_fy.index <= 1].sum())
print(d_fd_mean[d_fd_mean.index <= 50].sum())
print(d_fm_mean[d_fm_mean.index <= 1].sum())
print(s_duration[s_duration.index > 365].sum())
print(d_duration_mean[d_duration_mean.index > 30].sum())

0.759101035698238
0.5796301315196667
0.39338094422462727
0.27515208296667815
0.574037082548261
0.4381285066272672


In [3]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Dbedrock_Frequency', 'Dbedrock'),
    ('Area','area'),
    
    ('P/Sbedrock_div_Sr', 'Sbedrock'),
    ('P/Sbedrock_div_ET_mean', 'Sbedrock'),
    ('P/Sbedrock_div_PR_mean', 'Sbedrock'),
    ('P/ET_mean_div_PR_mean', 'et'),
    ('P/ET_mean_sub_Sbedrock_div_PR_mean', 'et'),
    ('P/Q_mean_div_PR_mean', 'tp'),
    ('P/PET_div_PR_mean', 'Band1')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['mask123'] == 1]
df = df[df['Sbedrock'] > 0]

dbedrcok_frequency = df.groupby('Dbedrock_Frequency')['Area'].sum()/1e12
print(dbedrcok_frequency)


# df.drop(labels='Sbedrock',axis=1,inplace=True)
df.drop(labels='mask123',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
with open('csv/Global_index.csv','w') as f:
    df.to_csv(f)
df

Dbedrock_Frequency
1.0     5.804232
2.0    13.474900
3.0     0.626150
4.0     0.001269
Name: Area, dtype: float64


,lat,lon,Sbedrock,Dbedrock_Frequency,Area,P/Sbedrock_div_Sr,P/Sbedrock_div_ET_mean,P/Sbedrock_div_PR_mean,P/ET_mean_div_PR_mean,P/ET_mean_sub_Sbedrock_div_PR_mean,P/Q_mean_div_PR_mean,P/PET_div_PR_mean
709889414,-55.764582,-67.439577,7.599648,2.0,120773.879698,21.041584,1.562866,0.745169,47.679657,46.934486,52.320343,52.082520
709889415,-55.764582,-67.435411,6.355453,2.0,120773.879698,17.660896,1.309530,0.623964,47.647930,47.023968,52.352070,51.997375
709889416,-55.764582,-67.431244,5.111259,2.0,120773.879698,14.255456,1.055210,0.502450,47.616127,47.113678,52.383877,52.677933
709975814,-55.760415,-67.439577,7.970362,2.0,120786.785891,21.041586,1.562866,0.772429,49.423893,48.651463,50.576107,48.822857
709975815,-55.760415,-67.435411,6.665475,2.0,120786.785891,17.660898,1.309530,0.646655,49.380722,48.734066,50.619278,48.762321
...,...,...,...,...,...,...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,39.297733,1.0,26309.225697,100.000000,36.635372,31.394192,85.693665,54.299473,14.306337,140.854782
3586500783,82.960426,-26.735409,39.880871,1.0,26309.225697,100.000000,37.097347,31.845638,85.843437,53.997799,14.156559,140.791473
3586500784,82.960426,-26.731242,40.464008,1.0,26309.225697,100.000000,37.557297,32.296680,85.993088,53.696407,14.006911,140.827164
3586500785,82.960426,-26.727075,41.047146,1.0,26309.225697,100.000000,38.015236,32.747311,86.142593,53.395283,13.857405,140.912659


In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area','area'),

    ('Dbedrock_median', 'Dbedrock'),
    ('D/Dbedrock_2011', 'Dbedrock'),
    ('D/Dbedrock_2014', 'Dbedrock'),
    ('D/Dbedrock_2015', 'Dbedrock'),
    ('D/Dbedrock_2017', 'Dbedrock')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df = df.dropna()
df = df[df['mask123'] == 1]
df = df[df['Sbedrock'] > 0]

dbedrock_median = df[df['Dbedrock_median'] > 0]['Dbedrock_median'].mean()
dbedrock_2011 = df[df['D/Dbedrock_2011'] > 0]['D/Dbedrock_2011'].mean()
dbedrock_2014 = df[df['D/Dbedrock_2014'] > 0]['D/Dbedrock_2014'].mean()
dbedrock_2015 = df[df['D/Dbedrock_2015'] > 0]['D/Dbedrock_2015'].mean()
dbedrock_2017 = df[df['D/Dbedrock_2017'] > 0]['D/Dbedrock_2017'].mean()

print(dbedrock_median)
print(dbedrock_2011)
print(dbedrock_2014)
print(dbedrock_2015)
print(dbedrock_2017)

df.drop(labels='Sbedrock',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask123',axis=1,inplace=True)
with open('csv/Global_Db.csv','w') as f:
    df.to_csv(f)
df

In [3]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),
    ('Koppen', 'Band1'),
    ('IGBP', 'LC', 0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

df['Koppen'] = df['Koppen'].round().astype(int)
df['IGBP'] = df['IGBP'].round().astype(int)

koppen_sum =  (df.groupby('Koppen')['Area'].sum()/1e12)/area_sum
igbp_sum =  (df.groupby('IGBP')['Area'].sum()/1e12)/area_sum

print(koppen_sum)
print(igbp_sum)

df.drop(labels='mask123',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
with open('csv/Global_igbp_koppen.csv','w') as f:
    df.to_csv(f)
df

Koppen
0     6.112033e-05
1     6.985340e-02
2     9.724314e-02
3     2.586658e-01
4     3.020918e-02
5     6.432178e-03
6     1.867198e-02
7     1.652807e-02
8     1.280205e-02
9     1.723371e-02
10    1.631729e-04
11    7.326664e-02
12    3.526883e-02
13    3.004124e-05
14    7.957056e-02
15    3.369053e-02
16    1.100464e-03
17    8.551537e-04
18    7.241484e-03
19    7.862298e-03
20    7.699848e-05
21    5.318061e-03
22    7.789368e-03
23    2.425886e-02
24    2.169995e-03
25    1.012745e-02
26    5.519520e-02
27    1.195051e-01
28    2.189220e-03
29    6.619355e-03
30    5.428513e-07
Name: Area, dtype: float64
IGBP
1    0.038585
2    0.226470
3    0.007160
4    0.069690
5    0.092616
6    0.002763
7    0.088304
8    0.191986
9    0.282426
Name: Area, dtype: float64


,lat,lon,Sbedrock,Area,Koppen,IGBP
709889414,-55.764582,-67.439577,7.599648,120773.879698,29,9
709889415,-55.764582,-67.435411,6.355453,120773.879698,29,9
709889416,-55.764582,-67.431244,5.111259,120773.879698,29,9
709889417,-55.764582,-67.427077,3.867064,120773.879698,29,9
709975814,-55.760415,-67.439577,7.970362,120786.785891,29,8
...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,39.297733,26309.225697,29,1
3586500783,82.960426,-26.735409,39.880871,26309.225697,29,1
3586500784,82.960426,-26.731242,40.464008,26309.225697,29,1
3586500785,82.960426,-26.727075,41.047146,26309.225697,29,1


In [2]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('Koppen', 'Band1'),
    ('IGBP', 'LC', 0),
    ('Aboveground', 'Band1'),
    ('Belowground', 'Band1'),
    ('DTB', 'Band1'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df['Aboveground_sum'] = df['Aboveground']*df['Area']
df['Belowground_sum'] = df['Belowground']*df['Area']

ag_sum = df['Aboveground_sum'].sum()
bg_sum = df['Belowground_sum'].sum()

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

ag_mask = df['Aboveground_sum'].sum()
bg_mask = df['Belowground_sum'].sum()

print(ag_mask/ag_sum)
print(bg_mask/bg_sum)

# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask123',axis=1,inplace=True)
df.drop(labels='Sbedrock',axis=1,inplace=True)
with open('csv/Global_statistics.csv','w') as f:
    df.to_csv(f)
df

0.3285835472991083
0.26498530051462754


,lat,lon,Area,Koppen,IGBP,Aboveground,Belowground,DTB,Aboveground_sum,Belowground_sum
709889414,-55.764582,-67.439577,120773.879698,29,9,278.0,95.0,0.0,3.357514e+07,1.147352e+07
709889415,-55.764582,-67.435411,120773.879698,29,9,102.0,56.0,0.0,1.231894e+07,6.763337e+06
709889416,-55.764582,-67.431244,120773.879698,29,9,73.0,46.0,100.0,8.816493e+06,5.555598e+06
709889417,-55.764582,-67.427077,120773.879698,29,9,208.0,93.0,100.0,2.512097e+07,1.123197e+07
709975814,-55.760415,-67.439577,120786.785891,29,8,580.0,186.0,0.0,7.005634e+07,2.246634e+07
...,...,...,...,...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,26309.225697,29,1,2.0,8.0,0.0,5.261845e+04,2.104738e+05
3586500783,82.960426,-26.735409,26309.225697,29,1,2.0,8.0,0.0,5.261845e+04,2.104738e+05
3586500784,82.960426,-26.731242,26309.225697,29,1,2.0,8.0,0.0,5.261845e+04,2.104738e+05
3586500785,82.960426,-26.727075,26309.225697,29,1,2.0,10.0,0.0,5.261845e+04,2.630923e+05


In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('DTB', 'Band1'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]

print(df['Area'].sum()/1e12)

# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask123',axis=1,inplace=True)
df.drop(labels='Sbedrock',axis=1,inplace=True)

with open('csv/Global_DTB.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
# print(area_sum)
# print(sbedrock_mean)
# print(sr_mean)
# print(ssoil_mean)

# print(area_DF_sum)

# print(s_fd)
# print(s_fy)
# print(d_fd_mean)
# print(d_fm_mean)
# print(s_duration)
# print(d_duration_mean)

# print(s_fd[s_fd.index <= 1000].sum())
# print(s_fy[s_fy.index <= 1].sum())
# print(d_fd_mean[d_fd_mean.index <= 50].sum())
# print(d_fm_mean[d_fm_mean.index <= 1].sum())
# print(s_duration[s_duration.index >= 365].sum())
# print(d_duration_mean[d_duration_mean.index >= 30].sum())

# print(dbedrcok_frequency)

# print(dbedrock_median)
# print(dbedrock_2011)
# print(dbedrock_2014)
# print(dbedrock_2015)
# print(dbedrock_2017)

# print(koppen_sum)
# print(igbp_sum)

# print(ag_mask/ag_sum)
# print(bg_mask/bg_sum)